In [129]:
import json
import pandas as pd
with open('data.json') as f:
    data = json.load(f)
data

{'random': '33',
 'random float': '7.662',
 'bool': 'true',
 'date': '1987-01-13',
 'regEx': 'hellooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo world',
 'enum': 'generator',
 'firstname': 'Vevay',
 'lastname': 'Kalinda',
 'city': 'Cairo',
 'country': 'Finland',
 'countryCode': 'TN',
 'email uses current data': 'Vevay.Kalinda@gmail.com',
 'email from expression': 'Vevay.Kalinda@yopmail.com',
 'array': ['Cacilie', 'Nerta', 'Blake', 'Wanda', 'Melanie'],
 'array of objects': [{'index': '0', 'index start at 5': '5'},
  {'index': '1', 'index start at 5': '6'},
  {'index': '2', 'index start at 5': '7'}],
 'Siana': {'age': '65'}}

In [130]:
def store_leaf_value(value, path, key=None):
    if key:
        data = [(path + '.' + key, value)]
    else:
        data = [(path, value)]
    dataframe = pd.DataFrame(data)
    dataframe.columns = ['Path', 'Value']
    return dataframe

store_leaf_value('age', '65', 'root.Siana')

,Path,Value
0,65.root.Siana,age


In [131]:
master_data_frame = pd.DataFrame()
def json_parser(data, master_data_frame, path):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, str):
                master_data_frame = pd.concat([master_data_frame, store_leaf_value(value, path, key)])
            elif isinstance(value, dict):
                dict_path = path + '.' + key
                master_data_frame = json_parser(data[key], master_data_frame, dict_path)
            elif isinstance(value, list):
                for i in range(len(value)):
                    list_path = path + '.' + key + '.' + str(i)
                    master_data_frame = json_parser(data[key][i], master_data_frame, list_path)
            else:
                pass
    elif isinstance(data, str):
        master_data_frame = pd.concat([master_data_frame, store_leaf_value(data, path, None)])

    return master_data_frame

all_data = json_parser(data, master_data_frame, 'root')
all_data.reset_index(drop=True, inplace=True)

In [132]:
all_data['Path'] = all_data['Path'].str.replace(r'\.\d+$', '.array_here', regex=True)
all_data['Path'] = all_data['Path'].str.replace(r'\.\d+\.', '.array_here.', regex=True)

result = all_data['Path'].value_counts().reset_index(name='Occurence')
result

,index,Occurence
0,root.array.array_here,5
1,root.array of objects.array_here.index start at 5,3
2,root.array of objects.array_here.index,3
3,root.random,1
4,root.country,1
5,root.email from expression,1
6,root.email uses current data,1
7,root.countryCode,1
8,root.city,1
9,root.random float,1


In [133]:
result['level'] = result['index'].str.count('\.')
result

,index,Occurence,level
0,root.array.array_here,5,2
1,root.array of objects.array_here.index start at 5,3,3
2,root.array of objects.array_here.index,3,3
3,root.random,1,1
4,root.country,1,1
5,root.email from expression,1,1
6,root.email uses current data,1,1
7,root.countryCode,1,1
8,root.city,1,1
9,root.random float,1,1
